# Data Transformation

Neste primeiro notebook, vamos inspecionar os dados disponibilizados pelo Spotify por meio do [Recsys Challenge 2018](https://recsys-challenge.spotify.com/) e transformá-los de sua estrutura atual em arquivos json para uma estrutura tabular pronta para processamentos seguintes.

@author: [Pedro Correia](https://github.com/pfcor)


In [1]:
import json
import os
import codecs
import datetime as dt
import pandas as pd
import tqdm
import mpd

Para iniciar as investigações, vamos observar o conteúdo (limpo e organizado) de uma playlist aleatória:

In [2]:
mpd.show_playlist(433)

=== 433 ===
Sad songs
  followers      1    
  modified       2017-04-16
  edits          9    
  collaborative  false
  num_artists    21   
  num_tracks     29   
  duration min   124.08

  # | track                          | album                                    | artist              
----------------------------------------------------------------------------------------------------
  0 | We Belong                      | Tropico                                  | Pat Benatar         
  1 | Please Forgive Me              | Australian Tour Edition 2013             | Bryan Adams         
  2 | Climax                         | Looking 4 Myself                         | Usher               
  3 | Battle Scars                   | Food & Liquor II: The Great American Ra  | Lupe Fiasco         
  4 | human                          | head or heart                            | Christina Perri     
  5 | Landslide                      | Fleetwood Mac                            | Fleetwood

Está presente na playlist acima todas as informações de fato disponíveis no dataset. Associada a cada uma das 1000000 de playlists temos o seu título, a quantidade de seguidores, a data de última edição e a quantidade de edições, se a playlist é colaborativa ou não, além das quantidades de artistas, faixas e de minutos de duração.

<hr>

O próximo passo é organizar essas informações em formato tabular, para utilização na sequência do trabalho.

In [5]:
def tabulate_playlist_data(input_path, output_path):
    
    playlist_features = [
        'pid',
        'name',
        'collaborative',  
        'modified_at', 
        'duration_ms',
        'num_tracks',
        'num_albums', 
        'num_artists',
        'num_followers',
        'num_edits'
    ]
    
    track_features = [
        'track_uri',
        'track_name',
        'album_uri',
        'album_name',
        'artist_uri',
        'artist_name',
        'duration_ms'
    ]

    slice_files = [
        os.sep.join((input_path, slice_file)) 
        for slice_file in 
        sorted(os.listdir(f'{input_path}'))
        if slice_file.startswith('mpd.slice')
    ]
    
    tracks = set() # accumulator for all tracks that appear
    
    for i, slice_file in tqdm.tqdm_notebook(enumerate(slice_files), total=len(slice_files), desc="Data Files"):
        
        # setting up accumulators
        playlist_metadata = [] # aggregating info about the playlist (name, num edits, num artists, duration...)
        track_metadata    = [] # aggregating info about the track (title, album, artist...)
        playlist_track    = [] # to-from playlist/track. also registering position of the track on the playlist here
        
        # reading file with playlists dataii
        with codecs.open(slice_file, 'r', 'utf-8') as f:
            playlists = json.loads(f.read())['playlists']
        
        # processing each playlist into tabular data
        for playlist in playlists:
            
            # appending lists with each of the relevant fields
            playlist_metadata.append([playlist[feat] for feat in playlist_features])
            
            # processing the tracks array of the current playlist
            for track in playlist['tracks']:
                
                # adding the connection of each playlist with each track
                playlist_track.append([playlist['pid'], track['track_uri'], track['pos']])
                
                # building a database of all tracks we see
                if track['track_uri'] not in tracks:
                    
                    # appending the info listed above
                    track_metadata.append([track[feat] for feat in track_features])
                    
                    # keeping record of all tracks already seen
                    tracks.add(track['track_uri'])
        
        # playlist metadata
        playlists_df = pd.DataFrame(playlist_metadata, columns=playlist_features)
        playlists_df['collaborative'] = playlists_df['collaborative'].map({'false': 0, 'true': 1})
        playlists_df.to_csv(os.sep.join((output_path, "playlists.csv")), index=False, header=(i==0), mode="w" if (i==0) else "a")
        
        # track data
        tracks_df = pd.DataFrame(track_metadata, columns=track_features)
        tracks_df.to_csv(os.sep.join((output_path, "tracks.csv")), index=False, header=(i==0), mode="w" if (i==0) else "a")
        
        # playlist-track
        playlist_track_df = pd.DataFrame(playlist_track, columns=['pid', 'track_uri', 'pos'])
        playlist_track_df.to_csv(os.sep.join((output_path, "playlist_track.csv")), index=False, header=(i==0), mode="w" if (i==0) else "a")


In [6]:
# calling the function to create the tabular files
input_path  = "data/original"
output_path = "data/formatted"

tabulate_playlist_data(input_path, output_path)

Vamos agora brevemente inspecionar os arquivos que acabamos de criar:

> playlists

In [3]:
df_playlists = pd.read_csv("data/formatted/playlists.csv")

In [4]:
df_playlists.shape

(1000000, 10)

Como esperado, são um milhão de playlists

In [5]:
df_playlists.head()

,pid,name,collaborative,modified_at,duration_ms,num_tracks,num_albums,num_artists,num_followers,num_edits
0,0,Throwbacks,0,1493424000,11532414,52,47,37,1,6
1,1,Awesome Playlist,0,1506556800,11656470,39,23,21,1,5
2,2,korean,0,1505692800,14039958,64,51,31,1,18
3,3,mat,0,1501027200,28926058,126,107,86,1,4
4,4,90s,0,1401667200,4335282,17,16,16,2,7


As informações estão registradas conforme esperado. Talvez caiba na sequência transformar os dados de `modified_at`e `duration_ms` em formatos de mais fácil entendimento, mas para o momento nossos objetivos foram alcançados.

In [6]:
# limpando namespace
del df_playlists

> tracks

aqui, vamos fazer algumas passos adicionais. vamos criar um id inteiro para as faixas e criar um dataset específico para os artistas:

In [11]:
df_tracks = pd.read_csv("data/formatted/tracks.csv")

In [44]:
df_tracks.shape

(2262292, 7)

In [45]:
df_tracks.head(3).T

,0,1,2
track_uri,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
track_name,Lose Control (feat. Ciara & Fat Man Scoop),Toxic,Crazy In Love
album_uri,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,spotify:album:25hVFAxTlDvXbx2X2QkUkE
album_name,The Cookbook,In The Zone,Dangerously In Love (Alben für die Ewigkeit)
artist_uri,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:artist:6vWDO969PvNqNYHIOW5v0m
artist_name,Missy Elliott,Britney Spears,Beyoncé
duration_ms,226863,198800,235933


In [46]:
# criando dataset de artistas
df_artists = df_tracks.drop_duplicates("artist_uri", keep="first")[["artist_uri", "artist_name"]]
df_artists = df_artists.reset_index().rename(columns={"index": "aid"})
df_artists.head()

,aid,artist_uri,artist_name
0,0,spotify:artist:2wIVse2owClT7go1WT98tk,Missy Elliott
1,1,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Britney Spears
2,2,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Beyoncé
3,3,spotify:artist:31TPClRtHm23RisEBtV3X7,Justin Timberlake
4,4,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Shaggy


In [47]:
df_artists.to_csv("data/formatted/artists.csv", index=False)

In [48]:
df_tracks = df_tracks.reset_index().rename(columns={"index": "tid"})
df_tracks = df_tracks.merge(df_artists[["aid", "artist_uri"]], on="artist_uri", how="left")

In [49]:
df_tracks.sample(3).T

,1786646,1038562,1304019
tid,1786646,1038562,1304019
track_uri,spotify:track:11ggz1zoh6J3g0fOnKHR1X,spotify:track:7gkI2b0MHzhvkSzuIch1Hj,spotify:track:0Ceo5IPMAG4YtVrxCKJc6r
track_name,One Chapter in the Book,Heaven,Espérame en el Cielo
album_uri,spotify:album:7umVRtgBbdU9Zj1K8a1o53,spotify:album:2TAlebSs2UfR5RJCknBezx,spotify:album:45CtwGkSFfWpBD9vK9JepV
album_name,What Makes a Man Start Fires?,"Apocalypse, Girl",La Sonora Radio Latina No. 2
artist_uri,spotify:artist:0z6zRFzl5njXWLVAisXQBz,spotify:artist:5eerVxtb9lY9p6D7m89Bs7,spotify:artist:6UtlGHaYcTwfQgBKqLaL0P
artist_name,Minutemen,Jenny Hval,Joe Valle Y Su Orquesta
duration_ms,62573,293243,185215
aid,184881,108788,1304019


In [50]:
df_artists[df_artists["aid"]==108788]

,aid,artist_uri,artist_name
24900,108788,spotify:artist:5eerVxtb9lY9p6D7m89Bs7,Jenny Hval


In [51]:
df_tracks.to_csv("data/formatted/tracks.csv", index=False)

In [52]:
# limpando namespace
del df_tracks

> playlist-track

In [4]:
df_pt = pd.read_csv("data/formatted/playlist_track.csv")

In [5]:
df_pt.shape

(66346428, 3)

In [3]:
df_tracks = df_tracks[["track_uri", "tid", "aid"]]

In [6]:
df_pt = df_pt.merge(df_tracks[["track_uri", "tid", "aid"]], on="track_uri", how="left").drop("track_uri", axis=1)[["pid", "tid", "aid", "pos"]]

In [8]:
df_pt.sample(5)

,pid,tid,aid,pos
17997348,342072,205360,2664,147
46564752,730807,12804,12804,45
44141990,699347,8861,327,126
47720195,747001,12905,29,27
435497,103531,60523,510,88


In [9]:
df_pt.to_csv("data/formatted/playlist_track.csv", index=False)

In [12]:
top_ten_tracks = df_pt["tid"].value_counts().head(20).reset_index().rename(columns={"index": "tid", "tid": "total"})
df_tracks[["tid", "track_name", "artist_name"]].merge(top_ten_tracks, on="tid").sort_values("total", ascending=False)

,tid,track_name,artist_name,total
4,1335,HUMBLE.,Kendrick Lamar,46574
16,3713,One Dance,Drake,43447
6,1396,Broccoli (feat. Lil Yachty),DRAM,41309
8,1874,Closer,The Chainsmokers,41079
14,2715,Congratulations,Post Malone,39987
13,2701,Caroline,Aminé,35202
3,1333,iSpy (feat. Lil Yachty),KYLE,35138
11,2698,Bad and Boujee (feat. Lil Uzi Vert),Migos,34999
1,1159,XO TOUR Llif3,Lil Uzi Vert,34922
15,3545,Location,Khalid,34657


<hr>